In [1]:
from mumax_helper import *

from scipy.signal import find_peaks

from pandas import read_csv
import matplotlib.pyplot as plt

In [3]:
# NUMERICAL PARAMETERS RELEVANT FOR THE SPECTRUM ANALYSIS
fmax = 1e9        # maximum frequency (in Hz) of the sinc pulse
T = 100e-9        # simulation time (longer -> better frequency resolution)
dt = .8e-10  # the sample time (Nyquist theorem taken into account)


# Note that this is a format string, this means that the statements inside the
# curly brackets get evaluated by python. In this way, we insert the values of
# the variables above in the script.

def simulation(freq):

    script = f"""
    nx   := 128
    cs := 205
    setgridsize(cs,cs,1)
    setcellsize(1e-9,1e-9,1e-9)

    domain := circle(200e-9).sub(circle(100e-9))
    setgeom(domain)

    Msat = 1e6
    Aex = 10e-12
    Dind = 2.2e-3
    Ku1 = 1e6
    AnisU = vector(0,0,1)
    alpha = 0.001

    B_ext = vector(0, 0, .03 * sin( 2*pi*{freq}*t)) 
    TableAdd(B_ext)
    TableAdd(torque)

    dia := 30e-9
    sep := 80e-9
    shape_1 := circle(dia).transl(0, -sep, 0)

    m = Uniform(0, 0, 1)
    m.setInShape(shape_1, NeelSkyrmion(1, -1).transl(0, -sep, 0 ))

    tableAdd(B_ext)
    tableAdd(ext_bubblespeed)

    minimize()
    autosave(m,{dt})
    // autosave(torque,{dt})
    tableautosave({dt})
    run({T})
    """

    return script


# run_mumax3(simulation(), "Circular Wire")


In [7]:
freq_list = np.linspace(5e8,6e10,15)

for freq in freq_list:
    title = f"Circular Wire f={freq}"
    run_mumax3(simulation(freq), title)

KeyboardInterrupt: 

In [4]:
arr = np.load("Circular Wire.out/m.npz")
def f(index):
    title = 'arr_'+str(index)
    m = arr[title]
    return m

for index in range(len(arr)):
    
    m0 = f(index)

    m0x = m0[0]
    m0y = m0[1]
    m0z = m0[2]

    dm0x_dx = np.diff(m0x, axis=1)[:, :, :-1]
    dm0y_dx = np.diff(m0y, axis=1)[:, :, :-1]
    dm0z_dx = np.diff(m0z, axis=1)[:, :, :-1]

    dm0x_dy = np.diff(m0x, axis=2)[:, :-1, :]
    dm0y_dy = np.diff(m0y, axis=2)[:, :-1, :]
    dm0z_dy = np.diff(m0z, axis=2)[:, :-1, :]

    dm0_dx = np.array([dm0x_dx, dm0y_dx, dm0z_dx])
    dm0_dy = np.array([dm0x_dy, dm0y_dy, dm0z_dy])
    cross_product = np.cross(dm0_dx, dm0_dy, axis=0)

    q_x = m0[0, :, :-1, :-1] * cross_product[0]
    q_y = m0[1, :, :-1, :-1] * cross_product[1]
    q_z = m0[2, :, :-1, :-1] * cross_product[2]
    q = q_x + q_y + q_z
    plt.imshow(q[0])
    plt.xlabel('nm')
    plt.ylabel('nm')
    label = r"Circular Wire.out/" + "q" + str(index) + ".png"
    plt.savefig(label, facecolor='white')


KeyboardInterrupt: 